In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
import xgboost as xgb
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm

from sklearn.preprocessing import StandardScaler

%matplotlib inline
%pylab inline

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [24]:
X_train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")

y_train = pd.DataFrame(X_train['is_duplicate'])
y_train.columns = ['y']
del X_train['is_duplicate']

skf = StratifiedKFold(5, shuffle=True, random_state=0)
folds = []
for itr, ite in skf.split(X_train, y_train.y):
    folds += [[itr, ite]]

In [68]:
X_train = pd.read_csv("data/train_inter.csv")

X_train['question1'] = X_train['question1'].apply(lambda x: str(x))
X_train['question2'] = X_train['question2'].apply(lambda x: str(x))

X_train['question1'] = X_train['question1'].apply(lambda x: x.lower().split())
X_train['question2'] = X_train['question2'].apply(lambda x: x.lower().split())



In [67]:
X_test = pd.read_csv("data/test_inter.csv")

X_test['question1'] = X_test['question1'].apply(lambda x: str(x))
X_test['question2'] = X_test['question2'].apply(lambda x: str(x))

X_test['question1'] = X_test['question1'].apply(lambda x: x.lower().split())
X_test['question2'] = X_test['question2'].apply(lambda x: x.lower().split())

In [69]:
X = map(lambda x: ' '.join(x), X_train['question1'])
X1 = map(lambda x: ' '.join(x), X_train['question2'])
Y = map(lambda x: ' '.join(x), X_test['question1'])
Y1 = map(lambda x: ' '.join(x), X_test['question2'])

hw = HashingVectorizer(n_features=200).fit(X_train['question1'] + X_train['question2'])


In [70]:
X = pd.DataFrame(hw.transform(X).todense())
X1 = pd.DataFrame(hw.transform(X1).todense())
Y = pd.DataFrame(hw.transform(Y).todense())
Y1 = pd.DataFrame(hw.transform(Y1).todense())

In [71]:
tfidf = feature_extraction.text.TfidfTransformer()
tfidf.fit(X+X1)

X = pd.DataFrame(tfidf.transform(X).todense())
X1 = pd.DataFrame(tfidf.transform(X1).todense())
Y = pd.DataFrame(tfidf.transform(Y).todense())
Y1 = pd.DataFrame(tfidf.transform(Y1).todense())

In [72]:
X_train = pd.concat([X_train, X], axis=1, join_axes=[X_train.index])
del X
X_train = pd.concat([X_train, X1], axis=1, join_axes=[X_train.index])
del X1
X_test = pd.concat([X_test, Y], axis=1, join_axes=[X_test.index])
del Y
X_test = pd.concat([X_test, Y1], axis=1, join_axes=[X_test.index])
del Y1

In [31]:
X_train[:2]

,id,qid1,qid2,question1,question2,word_match,tfidf_word_match,0,1,2,...,190,191,192,193,194,195,196,197,198,199
0,0,1,2,"[what, is, the, step, by, step, guide, to, inv...","[what, is, the, step, by, step, guide, to, inv...",0.727273,0.772164,0.0,0.0,0.0,...,0.0,0.0,0.325385,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,3,4,"[what, is, the, story, of, kohinoor, (koh-i-no...","[what, would, happen, if, the, indian, governm...",0.307692,0.361758,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
del X_train['question1']
del X_train['question2']
del X_train['qid1']
del X_train['qid2']
del X_train['id']

del X_test['question1']
del X_test['question2']
del X_test['test_id']

In [74]:
#tmp = abs(X_train.ix[:, 2:202] * X_train.ix[:, -200:])
#X_train = pd.concat([X_train, tmp], axis=1, join_axes=[X_train.index])
tmp = abs(X_train.ix[:, 2:202] - X_train.ix[:, -200:])
X_train = pd.concat([X_train, tmp], axis=1, join_axes=[X_train.index])

#tmp = abs(X_test.ix[:, 2:202] * X_test.ix[:, -200:])
#X_test = pd.concat([X_test, tmp], axis=1, join_axes=[X_test.index])
tmp = abs(X_test.ix[:, 2:202] - X_test.ix[:, -200:])
X_test = pd.concat([X_test, tmp], axis=1, join_axes=[X_test.index])

In [76]:
#X_train = pd.read_csv("data/train_inter.csv")
q_train = pd.read_csv("data/freq_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

f_train = pd.read_csv("data/train_features.csv", encoding = "ISO-8859-1")
f_train = f_train.ix[:, 2:]
X_train = pd.concat([X_train, f_train], axis=1)

In [143]:
del f_test

NameError: name 'f_test' is not defined

In [144]:
import gc
gc.collect()

80

In [99]:
q_test = pd.read_csv("data/freq_test.csv")


In [105]:
X_test = pd.concat([X_test, q_test], axis=1)

NameError: name 'q_test' is not defined

In [108]:
f_test = pd.read_csv("data/test_features.csv", encoding = "ISO-8859-1")
f_test = f_test.ix[:, 2:]

In [116]:
X_test = pd.concat([X_test, f_test], axis=1)

In [118]:
X_test[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance,skew_q1vec,skew_q2vec,kur_q1vec,kur_q2vec
0,0.266667,0.274019,0.0,0.0,0.0,0.0,0.0,-0.156312,0.0,0.0,...,12.006720,1.0,164.254264,0.889327,0.407153,0.483565,0.045990,0.009958,0.039938,-0.144866
1,0.500000,0.480962,0.0,0.0,0.0,0.0,0.0,-0.164904,0.0,0.0,...,6.501658,1.0,115.512012,0.466698,0.210239,0.245248,-0.017419,-0.046821,0.207580,-0.042937


In [39]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 1100

#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.8,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.1,
    'min_child_weight': 5,
    #'max_depth': 6
    #'num_class': 3
    
}

pred_train = np.zeros(len(y_train))
xgbs = []
sc,sc_mean = [],[]

for itr, ite in folds:
    ypred = []
    for i in range(1):
               
        param['seed'] = i+1
        
        #xgboost
        '''
        Xdatatrain = xgb.DMatrix(data=X_train.ix[itr, :].values,
                                     label=y_train.ix[itr].values)
        Xdataval = xgb.DMatrix(data=X_train.ix[ite, :].values,
                                     label=y_train.ix[ite].values)

        plst = list(param.items())
        watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]
        bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)

        
        #rc = ensemble.ExtraTreesClassifier(n_estimators=1300, criterion='gini', max_depth=None, n_jobs=-1)
        #pred_train[ite] = bst.predict(Xdataval)
        #neigh = KNeighborsClassifier(n_neighbors=2)
        #neigh.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = bst.predict(Xdataval)
        #ypred = neigh.predict_proba(X_train.ix[ite, :])
        xgbs.append(bst)
        '''
        
        # train
        lgb = lgbm.sklearn.LGBMClassifier(n_estimators=300, seed=0, **params)
        lgb.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = lgb.predict_proba(X_train.ix[ite, :])[:, 1]

        '''
        model.fit(X_train.ix[itr, :].values, y_train_cat[itr],
            epochs=60,
            batch_size=1000)
        ypred = model.predict(X_train.ix[ite, :].values)
        '''
    #ypred = sum(ypred) / len(ypred) 0.401408 0.392476
    pred_train[ite] = ypred
    
    
    sc.append(log_loss(y_train.ix[ite, :], pred_train[ite]))

    
print('XGB: {:.3f} +- {:.3f}'.format(np.mean(sc), np.std(sc)))

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

XGB: 0.291 +- 0.001


In [40]:
def foo(x):
    a = 0.165 / 0.37
    b = (1 - 0.165) / (1 - 0.37) 
    return  a * x / (a * x + b * (1 - x))

In [41]:
pred_train = foo(pred_train)

In [42]:
pred_train = pd.DataFrame(pred_train)
pred_train.columns = ['y']
pred_train.to_csv("stacking/lgb_14.csv", index=None)

In [158]:
submit = pd.read_csv("data/sample_submission.csv")

In [43]:
lgb = lgbm.sklearn.LGBMClassifier(n_estimators=300, seed=0, **params)
lgb.fit(X_train, y_train)


/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.1, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=5,
        min_split_gain=0, n_estimators=300, nthread=-1, num_leaves=31,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [142]:
X_test = X_test.ix[1000001:, :]

In [145]:
test_pred2 = lgb.predict_proba(X_test)[:, 1]

In [126]:
test_pred1[:4]

array([ 0.00778551,  0.58706412,  0.45319938,  0.00166404])

In [127]:
test_pred1[-4:]

array([  4.85291907e-04,   2.61291438e-01,   6.26312391e-01,
         1.66797416e-03])

In [149]:
test_pred2[:4]

array([  3.51530930e-01,   3.07114939e-04,   1.03535362e-03,
         9.95933188e-02])

In [151]:
test_pred2[-4:]

array([  1.67587699e-03,   2.31711933e-04,   3.72899384e-01,
         3.89707027e-01])

In [148]:
len(test_pred1)

1000001

In [152]:
len(test_pred2)

1345795

In [153]:
test_pred = np.concatenate([test_pred1, test_pred2], axis=0)

In [154]:
len(test_pred)

2345796

In [155]:
test_pred

array([  7.78551300e-03,   5.87064116e-01,   4.53199376e-01, ...,
         2.31711933e-04,   3.72899384e-01,   3.89707027e-01])

In [146]:
del pred_train

NameError: name 'pred_train' is not defined

In [156]:
import gc
gc.collect()

0

In [13]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 1100

In [ ]:
Xdatatrain = xgb.DMatrix(data=X_train.values,
                                     label=y_train.values)
Xdataval = xgb.DMatrix(data=X_train.values,
                                     label=y_train.values)
Xdatatest = xgb.DMatrix(data=X_test.values)


plst = list(param.items())
watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]
bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)


test_pred = bst.predict(Xdatatest)


In [20]:
bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)


[0]	train-logloss:0.657056	eval-logloss:0.657056
[15]	train-logloss:0.464559	eval-logloss:0.464559
[30]	train-logloss:0.426999	eval-logloss:0.426999
[45]	train-logloss:0.412911	eval-logloss:0.412911
[60]	train-logloss:0.404036	eval-logloss:0.404036
[75]	train-logloss:0.396529	eval-logloss:0.396529
[90]	train-logloss:0.39033	eval-logloss:0.39033
[105]	train-logloss:0.384979	eval-logloss:0.384979
[120]	train-logloss:0.380431	eval-logloss:0.380431
[135]	train-logloss:0.375948	eval-logloss:0.375948
[150]	train-logloss:0.371985	eval-logloss:0.371985
[165]	train-logloss:0.368135	eval-logloss:0.368135
[180]	train-logloss:0.364906	eval-logloss:0.364906
[195]	train-logloss:0.361831	eval-logloss:0.361831
[210]	train-logloss:0.358712	eval-logloss:0.358712
[225]	train-logloss:0.355959	eval-logloss:0.355959
[240]	train-logloss:0.353491	eval-logloss:0.353491
[255]	train-logloss:0.350956	eval-logloss:0.350956
[270]	train-logloss:0.348754	eval-logloss:0.348754
[285]	train-logloss:0.346606	eval-logloss

In [ ]:
test_pred = bst.predict(Xdatatest)


In [157]:
test_pred = foo(test_pred)

In [159]:
submit['is_duplicate'] = test_pred

In [160]:
submit.to_csv("stacking/lgb_14_test.csv", index=None)

In [161]:
submit

,test_id,is_duplicate
0,0,0.002633
1,1,0.323567
2,2,0.218058
3,3,0.000561
4,4,0.095342
5,5,0.007831
6,6,0.956379
7,7,0.098221
8,8,0.363882
9,9,0.002275
